## Eigene Umgebung

Autofahrer

Soll die Geschwindigkeit immer zw. 45km/h und 55km/h halten

Drei Actions: Beschleunigen, Bremsen, Nichts tun

Während des Prozesses wird das Auto automatisch langsamer/schneller (Bergauf/Bergab)

In [11]:
import gymnasium as gym
import numpy as np
from typing import Optional
from gymnasium.spaces import Discrete, Box

In [12]:
class CarDriver(gym.Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(0, 100, dtype=np.int8)
        self.state = 50
        self.driveLength = 60

    def step(self, action):
        if action == 1:
            self.state += 3
        elif action == 2:
            self.state -= 3

        if self.state >= 45 and self.state <= 55:
            reward = 1
        else:
            reward = 0

        self.state += np.random.randint(3) - 1
        self.driveLength -= 1

        return np.array(self.state), reward, self.driveLength <= 0, False, {}
    
    def reset(self, *, seed: Optional[int] = None, options: Optional[dict] = None):
        self.state = 50
        self.driveLength = 60
        return (np.array(self.state), {})

In [13]:
env = CarDriver()

In [15]:
durchgaenge = 10
for d in range(durchgaenge):
    start = env.reset()  # Am Anfang von jedem Durchgang wird die Umgebung zurückgesetzt
    score = 0  # Summiert die Belohnung nach jedem Step auf
    done = False

    while not done:  # Führe die Umgebung aus, bis die step Methode True zurück gibt
        action = env.action_space.sample()  # Wähle eine zufällige Action, und füge diese bei step ein
        newState, reward, done, term, info = env.step(action)  # Führe die Umgebung einmal aus
        score += reward
    print(f"Durchgang: {d}, Score: {score}")

Durchgang: 0, Score: 8
Durchgang: 1, Score: 16
Durchgang: 2, Score: 16
Durchgang: 3, Score: 39
Durchgang: 4, Score: 19
Durchgang: 5, Score: 2
Durchgang: 6, Score: 15
Durchgang: 7, Score: 8
Durchgang: 8, Score: 6
Durchgang: 9, Score: 16


In [16]:
from stable_baselines3 import PPO
env = CarDriver()
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=20_000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | 21.4     |
| time/              |          |
|    fps             | 795      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 21.5        |
| time/                   |             |
|    fps                  | 458         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.005709998 |
|    clip_fraction        | 0.0242      |
|    clip_range           | 0.2         |
|    entropy_loss   

In [17]:
model.save("Models/CarDriver")

C:\Users\lk3\AppData\Local\Programs\Python\Python312\Lib\site-packages\stable_baselines3\common\save_util.py:284: UserWarning: Path 'Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [20]:
env = CarDriver()

In [21]:
model = PPO.load("Models/CarDriver", env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [23]:
durchgaenge = 5
for d in range(durchgaenge):
    state = env.reset()  # Am Anfang von jedem Durchgang wird die Umgebung zurückgesetzt
    score = 0  # Summiert die Belohnung nach jedem Step auf
    done = False
    
    while not done:  # Führe die Umgebung aus, bis die step Methode True zurück gibt
        if len(state) == 2:
            action, _ = model.predict(state[0])
        else:
            action, _ = model.predict(state)
        state, reward, done, term, info = env.step(action)  # Führe die Umgebung einmal aus
        score += reward
    print(f"Durchgang: {d}, Score: {score}")

ValueError: You have passed a tuple to the predict() function instead of a Numpy array or a Dict. You are probably mixing Gym API with SB3 VecEnv API: `obs, info = env.reset()` (Gym) vs `obs = vec_env.reset()` (SB3 VecEnv). See related issue https://github.com/DLR-RM/stable-baselines3/issues/1694 and documentation for more information: https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecenv-api-vs-gym-api